In [37]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.types import StringType,DoubleType,FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import regexp_replace
import locale
import sys
import os
import re

In [38]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [39]:
spark

# Leyendo lista de Colegios y Estudiantes

In [40]:
base = '../Colegios'
base2 = '../SB'
colegios = os.listdir('../Colegios')
estudiantes = os.listdir('../SB')

In [41]:
estudiantes.sort()
colegios.sort()
estuDic = {}
coleDic = {}
for estu in estudiantes:
    estuDic[re.findall('\d+', estu)[1]] = estu

for colegio in colegios:
    coleDic[re.findall('\d+', colegio)[1]] = colegio

In [42]:
dfestu = {}
dfcole = {}
for i in coleDic:
    dfestu[i] = None
    dfcole[i] = None

In [43]:
# Funciòn para unir cualquier nùmero de dataframes con la misma estructura 
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [44]:
print("Archivos de estudiantes: ", estuDic['20071'],estuDic['20072'])
print("Archivos de colegios: ",coleDic['2007'])

Archivos de estudiantes:  SB11_20071.txt SB11_20072.txt
Archivos de colegios:  SB11-CLASIFI-PLANTELES-2007.txt


In [45]:
columnasPuntos = [
 'cole_cod_dane_institucion',
 'cole_nombre_sede',
 'punt_lenguaje',
 'punt_matematicas',
 'punt_c_sociales',
 'punt_filosofia',
 'punt_biologia',
 'punt_quimica',
 'punt_fisica',
 'punt_ingles',
 'punt_profundizacion',
 'punt_interdisciplinar']

In [46]:
# SE leen los dos archivos y se crean dos dataframes
df1 = spark.read.load(base2 + '/' + estuDic['20071'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬').select(columnasPuntos)
df2 = spark.read.load(base2 + '/' + estuDic['20072'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬').select(columnasPuntos)

In [47]:
df1.columns

['cole_cod_dane_institucion',
 'cole_nombre_sede',
 'punt_lenguaje',
 'punt_matematicas',
 'punt_c_sociales',
 'punt_filosofia',
 'punt_biologia',
 'punt_quimica',
 'punt_fisica',
 'punt_ingles',
 'punt_profundizacion',
 'punt_interdisciplinar']

In [48]:
# SE unen los dos dataframes
columnasPuntosc = [
 'punt_lenguaje',
 'punt_matematicas',
 'punt_c_sociales',
 'punt_filosofia',
 'punt_biologia',
 'punt_quimica',
 'punt_fisica',
 'punt_ingles',
 'punt_profundizacion',
 'punt_interdisciplinar']
df = unionAll(df1,df2)
for cp in columnasPuntosc:
    df = df.withColumn(cp, regexp_replace(cp, ',', '.'))
    df = df.withColumn(cp, df[cp].cast("float"))

df = df.na.fill({'cole_cod_dane_institucion':'--'})
df = df.na.fill({'cole_nombre_sede':'--'})
df1 = df2 = None

In [49]:
# SE evalua sí tiene valores nullos
print("Nùmero de filas con cole_cod_dane_institucion nulos: ", df.filter("cole_cod_dane_institucion = '--'").count())
print("Nùmero de filas con cole_nombre_sede nulos: ", df.filter("cole_nombre_sede = '--'").count())

Nùmero de filas con cole_cod_dane_institucion nulos:  6981
Nùmero de filas con cole_nombre_sede nulos:  0


In [50]:
print(df.printSchema())

root
 |-- cole_cod_dane_institucion: string (nullable = false)
 |-- cole_nombre_sede: string (nullable = false)
 |-- punt_lenguaje: float (nullable = true)
 |-- punt_matematicas: float (nullable = true)
 |-- punt_c_sociales: float (nullable = true)
 |-- punt_filosofia: float (nullable = true)
 |-- punt_biologia: float (nullable = true)
 |-- punt_quimica: float (nullable = true)
 |-- punt_fisica: float (nullable = true)
 |-- punt_ingles: float (nullable = true)
 |-- punt_profundizacion: float (nullable = true)
 |-- punt_interdisciplinar: float (nullable = true)

None


In [51]:
# Contando el numero de estudiantes por instituciòn y mostrandola de forma descendente
dsCountEst = df.groupBy('cole_nombre_sede').count().orderBy('count',ascending=
False)

In [52]:
dsCountEst = dsCountEst.withColumnRenamed("count","num_est")
dsCountEst.show(10,False)

+---------------------------------------------------+-------+
|cole_nombre_sede                                   |num_est|
+---------------------------------------------------+-------+
|INST EDUC CEFA                                     |1088   |
|INEM FRANCISCO DE PAULA SANTANDER                  |1013   |
|INSTITUTO TECNICO INDUSTRIAL                       |875    |
|INEM SANTIAGO PEREZ                                |871    |
|INST EDUC INEM JOSE FELIX DE RESTREPO              |856    |
|JORGE ELIECER GAITAN                               |750    |
|COL DIST FERNANDO MAZUERA VILLEGAS                 |684    |
|LIC SALAZAR Y HERRERA                              |669    |
|SEDE A - I E TEC INEM CUSTODIO GARCÍA ROVIRA       |660    |
|INSTITUCION EDUCATIVA JOSE MANUEL RODRIGUEZ TORICES|648    |
+---------------------------------------------------+-------+
only showing top 10 rows



In [53]:
rowmax = dsCountEst.agg({"num_est": "max"}).collect()[0]
rowmin = dsCountEst.agg({"num_est": "min"}).collect()[0]

num_est_max = rowmax["max(num_est)"]
num_est_min = rowmin["min(num_est)"]

print("Numero màximo de estudiantes: ",num_est_max)
print("Numero mìnimo de estudiantes: ",num_est_min)

Numero màximo de estudiantes:  1088
Numero mìnimo de estudiantes:  1


In [54]:
#Se crea una columna con el total de los puntajes   
df= df.withColumn("Suma",df['punt_lenguaje']+df['punt_matematicas']+df['punt_c_sociales']+
                     df['punt_filosofia']+df['punt_biologia']+df['punt_quimica']+df['punt_fisica']+
                     df['punt_ingles']+df['punt_profundizacion']+df['punt_interdisciplinar'])

## Resultados finales por instituciòn

In [55]:
# Còdigo para obtener la media de los resultados de las evaluaciones de todos los estudiantes por instituciòn 
# y posiciòn en orden descendente.
from pyspark.sql.functions import *
from pyspark.sql.window import Window

#df = df.groupBy('cole_cod_dane_institucion','cole_nombre_sede').mean()
df = df.groupBy('cole_nombre_sede').mean()

rowprommax = df.agg({"avg(Suma)": "max"}).collect()[0]
rowprommin = df.agg({"avg(Suma)": "min"}).collect()[0]

prom_est_max = rowprommax["max(avg(Suma))"]
prom_est_min = rowprommin["min(avg(Suma))"]

print("Màxima calificaciòn en colegios: ", prom_est_max)
print("Mìxima calificaciòn en colegios: ", prom_est_min)

Màxima calificaciòn en colegios:  578.0105333830181
Mìxima calificaciòn en colegios:  312.5690002441406


## Creación de Indices de evaluación
$Indice\_Prom = \frac{avg(Suma)}{max(avg(Suma))}$&nbsp;&nbsp;
$0 < Indice\_Prom \leq 1$

$Indice\_Num = \frac{num\_est}{max(num\_est)}$&nbsp;&nbsp;
$0 < Indice\_Num \leq 1$

$Indice\_Total = Indice\_Prom \times Indice\_Num$&nbsp;&nbsp;
$0 < Indice\_Total \leq 1 $

$Indice\_Total$: tiene en consideraciòn el promedio de las calificaciones y el nùmero de estudiantes 

In [56]:
dfjoin = df.join(dsCountEst,"cole_nombre_sede")
dfjoin= dfjoin.withColumn("Indice_Prom",round(dfjoin['avg(Suma)']/prom_est_max,3))
dfjoin= dfjoin.withColumn("Indice_Num",round(dfjoin['num_est']/num_est_max,3))
dfjoin= dfjoin.withColumn("Indice_Total",round(dfjoin['Indice_Prom']*dfjoin['Indice_Num'],3))

In [57]:
dfjoin.printSchema()

root
 |-- cole_nombre_sede: string (nullable = false)
 |-- avg(punt_lenguaje): double (nullable = true)
 |-- avg(punt_matematicas): double (nullable = true)
 |-- avg(punt_c_sociales): double (nullable = true)
 |-- avg(punt_filosofia): double (nullable = true)
 |-- avg(punt_biologia): double (nullable = true)
 |-- avg(punt_quimica): double (nullable = true)
 |-- avg(punt_fisica): double (nullable = true)
 |-- avg(punt_ingles): double (nullable = true)
 |-- avg(punt_profundizacion): double (nullable = true)
 |-- avg(punt_interdisciplinar): double (nullable = true)
 |-- avg(Suma): double (nullable = true)
 |-- num_est: long (nullable = false)
 |-- Indice_Prom: double (nullable = true)
 |-- Indice_Num: double (nullable = true)
 |-- Indice_Total: double (nullable = true)



In [58]:
dfjoin = dfjoin.select('cole_nombre_sede','avg(Suma)','num_est','Indice_Prom','Indice_Num','Indice_Total')
dfjoin = dfjoin.withColumn("rankProm", dense_rank().over(Window.orderBy(desc("Indice_Prom"))))
dfjoin = dfjoin.withColumn("rankNum", dense_rank().over(Window.orderBy(desc("Indice_Num"))))
dfjoin = dfjoin.withColumn("rankTotal", dense_rank().over(Window.orderBy(desc("Indice_Total"))))
dfjoin = dfjoin.withColumn("Media",round(dfjoin['avg(Suma)'],2))
dfjoin.printSchema()

root
 |-- cole_nombre_sede: string (nullable = false)
 |-- avg(Suma): double (nullable = true)
 |-- num_est: long (nullable = false)
 |-- Indice_Prom: double (nullable = true)
 |-- Indice_Num: double (nullable = true)
 |-- Indice_Total: double (nullable = true)
 |-- rankProm: integer (nullable = true)
 |-- rankNum: integer (nullable = true)
 |-- rankTotal: integer (nullable = true)
 |-- Media: double (nullable = true)



In [59]:
# Se ordena por Indice_Prom
dfjoin = dfjoin.orderBy('rankProm',ascending= True)
dfjoin.select('cole_nombre_sede','Media','Indice_Prom','num_est','rankProm','rankNum','rankTotal').show(10)

+--------------------+------+-----------+-------+--------+-------+---------+
|    cole_nombre_sede| Media|Indice_Prom|num_est|rankProm|rankNum|rankTotal|
+--------------------+------+-----------+-------+--------+-------+---------+
|INST ALBERTO MERA...|578.01|        1.0|     19|       1|    354|      271|
|LIC NAVARRA      ...| 576.3|      0.997|     44|       2|    331|      248|
|LIC  DE CERVANTES...|571.21|      0.988|     93|       3|    286|      204|
|COL REFOUS       ...|564.17|      0.976|     54|       4|    321|      239|
|ASPAEN GIMNASIO C...| 562.5|      0.973|     23|       5|    350|      268|
|CENT. DE ENSEÑANZ...|562.33|      0.973|      5|       5|    366|      283|
|COLEGIO SEMINARIO...|561.91|      0.972|     27|       6|    346|      264|
|COL CORAZONISTA H...|560.27|      0.969|     79|       7|    298|      217|
|LIC CAMPO DAVID  ...|558.77|      0.967|     16|       8|    356|      273|
|COL  SAN BONIFACI...| 556.2|      0.962|     16|       9|    356|      274|

In [60]:
dfjoin = dfjoin.orderBy('rankNum',ascending= True)
dfjoin.select('cole_nombre_sede','Media','Indice_Num','num_est','rankProm','rankNum','rankTotal').show(10)

+--------------------+------+----------+-------+--------+-------+---------+
|    cole_nombre_sede| Media|Indice_Num|num_est|rankProm|rankNum|rankTotal|
+--------------------+------+----------+-------+--------+-------+---------+
|      INST EDUC CEFA|424.49|       1.0|   1088|     198|      1|        1|
|INEM FRANCISCO DE...|412.06|     0.931|   1013|     219|      2|        2|
|INSTITUTO TECNICO...|410.13|     0.804|    875|     222|      3|        4|
| INEM SANTIAGO PEREZ|411.77|     0.801|    871|     220|      4|        5|
|INST EDUC INEM JO...| 422.5|     0.787|    856|     201|      5|        3|
|JORGE ELIECER GAITAN|395.79|     0.689|    750|     247|      6|        6|
|COL DIST FERNANDO...|410.61|     0.629|    684|     222|      7|        8|
|LIC SALAZAR Y HER...|440.94|     0.615|    669|     169|      8|        7|
|SEDE A - I E TEC ...| 419.0|     0.607|    660|     207|      9|        9|
|INSTITUCION EDUCA...|397.47|     0.596|    648|     244|     10|       10|
+-----------

In [61]:
# Se ordena por Indice_Total
dfjoin = dfjoin.orderBy('rankTotal',ascending= True)
dfjoin.select('cole_nombre_sede','Media','Indice_Total','num_est','rankProm','rankNum','rankTotal').show(10)

+--------------------+------+------------+-------+--------+-------+---------+
|    cole_nombre_sede| Media|Indice_Total|num_est|rankProm|rankNum|rankTotal|
+--------------------+------+------------+-------+--------+-------+---------+
|      INST EDUC CEFA|424.49|       0.734|   1088|     198|      1|        1|
|INEM FRANCISCO DE...|412.06|       0.664|   1013|     219|      2|        2|
|INST EDUC INEM JO...| 422.5|       0.575|    856|     201|      5|        3|
|INSTITUTO TECNICO...|410.13|       0.571|    875|     222|      3|        4|
| INEM SANTIAGO PEREZ|411.77|        0.57|    871|     220|      4|        5|
|JORGE ELIECER GAITAN|395.79|       0.472|    750|     247|      6|        6|
|LIC SALAZAR Y HER...|440.94|       0.469|    669|     169|      8|        7|
|COL DIST FERNANDO...|410.61|       0.447|    684|     222|      7|        8|
|SEDE A - I E TEC ...| 419.0|        0.44|    660|     207|      9|        9|
|COL DE LA UNIV PO...|470.79|        0.44|    589|     118|     

## Colegios

In [19]:
# se crea un dataframe con una columna del año.
from pyspark.sql.functions import lit
df = df.withColumn('año',lit(2007))
df.show(5,False)

+-------------------------+------------------+---------------------+--------------------+-------------------+------------------+------------------+------------------+-----------------+------------------------+--------------------------+-----------------+----+----+
|cole_cod_dane_institucion|avg(punt_lenguaje)|avg(punt_matematicas)|avg(punt_c_sociales)|avg(punt_filosofia)|avg(punt_biologia)|avg(punt_quimica) |avg(punt_fisica)  |avg(punt_ingles) |avg(punt_profundizacion)|avg(punt_interdisciplinar)|avg(Suma)        |rank|año |
+-------------------------+------------------+---------------------+--------------------+-------------------+------------------+------------------+------------------+-----------------+------------------------+--------------------------+-----------------+----+----+
|311001089221             |61.91473629600123 |70.93947340312756    |67.34420977140728   |51.50736838892887  |65.5378946003161  |58.80894711143092 |58.67578908016807 |76.65473837601512|6.891579000573409    

In [20]:
# SE crea Dataframe de colegios 2007
dfc = spark.read.load(base + '/' + coleDic['2007'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='\t')
dfc = dfc.withColumn('COLE_CODIGO_COLEGIO', dfc['COLE_CODIGO_COLEGIO'].cast("string"))
dfc = dfc.withColumn('COLE_CODMPIO_COLEGIO', dfc['COLE_CODMPIO_COLEGIO'].cast("string"))

In [21]:
print(dfc.printSchema())

root
 |-- COLE_CODIGO_COLEGIO: string (nullable = true)
 |-- COLE_INST_NOMBRE: string (nullable = true)
 |-- COLE_CODMPIO_COLEGIO: string (nullable = true)
 |-- COLE_MPIO_COLEGIO: string (nullable = true)
 |-- COLE_DEPTO_COLEGIO: string (nullable = true)
 |-- COLE_INST_JORNADA: string (nullable = true)
 |-- COLE_CALENDARIO_COLEGIO: string (nullable = true)
 |-- COLE_GENEROPOBLACION: string (nullable = true)
 |-- COLE_NATURALEZA: string (nullable = true)
 |-- COLE_CIENCIAS_SOCIALES: integer (nullable = true)
 |-- COLE_QUIMICA: integer (nullable = true)
 |-- COLE_FISICA: integer (nullable = true)
 |-- COLE_BIOLOGIA: integer (nullable = true)
 |-- COLE_FILOSOFIA: integer (nullable = true)
 |-- COLE_MATEMATICAS: integer (nullable = true)
 |-- COLE_LENGUAJE: integer (nullable = true)
 |-- COLE_INGLES: integer (nullable = true)
 |-- COLE_GEOGRAFIA: string (nullable = true)
 |-- COLE_HISTORIA: string (nullable = true)
 |-- COLE_CATEGORIA: string (nullable = true)
 |-- COLE_ESTUDIANTES_PRESENT

In [22]:
colsColegio = ['COLE_CODIGO_COLEGIO',
 'COLE_CODMPIO_COLEGIO',
 'COLE_CIENCIAS_SOCIALES',
 'COLE_QUIMICA',
 'COLE_FISICA',
 'COLE_BIOLOGIA',
 'COLE_FILOSOFIA',
 'COLE_MATEMATICAS',
 'COLE_LENGUAJE',
 'COLE_INGLES',
 'COLE_GEOGRAFIA',
 'COLE_HISTORIA']

In [23]:
dfc = dfc.select(colsColegio)

In [24]:
dfc.show(5,False)

+-------------------+--------------------+----------------------+------------+-----------+-------------+--------------+----------------+-------------+-----------+--------------+-------------+
|COLE_CODIGO_COLEGIO|COLE_CODMPIO_COLEGIO|COLE_CIENCIAS_SOCIALES|COLE_QUIMICA|COLE_FISICA|COLE_BIOLOGIA|COLE_FILOSOFIA|COLE_MATEMATICAS|COLE_LENGUAJE|COLE_INGLES|COLE_GEOGRAFIA|COLE_HISTORIA|
+-------------------+--------------------+----------------------+------------+-----------+-------------+--------------+----------------+-------------+-----------+--------------+-------------+
|135442             |47001               |5                     |6           |5          |7            |4             |7               |8            |5          |null          |null         |
|135459             |25322               |6                     |7           |6          |6            |5             |7               |6            |5          |null          |null         |
|135467             |5001               

In [25]:
print("El numero de colegios: ", dfc.select('COLE_CODIGO_COLEGIO').count())

El numero de colegios:  9680


## Anàlisis comparativo entre estudiantes y colegios

In [26]:
dfjoin = df.join(dfc, df.cole_cod_dane_institucion == dfc.COLE_CODIGO_COLEGIO)

In [27]:
print("Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios:", dfjoin.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios: 0


In [28]:
dfjoin = df.join(dfc, df.cole_cod_dane_institucion == dfc.COLE_CODMPIO_COLEGIO)
dfjoin.count()
print("Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios:", dfjoin.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios: 0


La informaciòn de colegios no coincide con la informaciòn de estudiantes, no existe cole_cod_dane_institucion en el archivo de colegios.